# Bedrock AgentCore Gatewayを使用してOpenAPI APIをMCPツールに変換する

## 概要
お客様はJSON または YAML形式のOpenAPI仕様を持参し、Bedrock AgentCore Gatewayを使用してAPIをMCPツールに変換できます。ここでは、APIキーを使用してNASAのOpen APIを呼び出す火星天気エージェントの構築を実演します。

Gatewayワークフローでは、エージェントを外部ツールに接続するために以下のステップが含まれます：
* **Gatewayのツールを作成** - REST API用のOpenAPI仕様などのスキーマを使用してツールを定義します。OpenAPI仕様はGateway作成のためにAmazon Bedrock AgentCoreによって解析されます。
* **Gatewayエンドポイントを作成** - インバウンド認証を伴うMCPエントリーポイントとして機能するGatewayを作成します。
* **Gatewayにターゲットを追加** - Gatewayが特定のツールへのリクエストをルーティングする方法を定義するOpenAPIターゲットを設定します。OpenAPIファイルに含まれるすべてのAPIがMCP互換ツールとなり、Gateway エンドポイントURL経由で利用できるようになります。各OpenAPI Gatewayターゲットのアウトバウンド認証を設定します。
* **エージェントコードを更新** - エージェントをGatewayエンドポイントに接続し、統合されたMCPインターフェースを通じてすべての設定済みツールにアクセスします。

![仕組み](images/openapi-gateway-apikey.png)

### チュートリアル詳細


| 情報                 | 詳細                                                      |
|:---------------------|:----------------------------------------------------------|
| チュートリアル形式   | インタラクティブ                                          |
| AgentCoreコンポーネント | AgentCore Gateway, AgentCore Identity                     |
| エージェントフレームワーク | Strands Agents                                            |
| Gatewayターゲット形式  | OpenAPI                                                   |
| エージェント         | 火星天気エージェント                                      |
| インバウンド認証IdP  | Amazon Cognito                                            |
| アウトバウンド認証   | APIキー                                                   |
| LLMモデル            | Anthropic Claude Sonnet 3.7, Amazon Nova Pro              |
| チュートリアルコンポーネント | AgentCore Gatewayの作成とAgentCore Gatewayの呼び出し |
| チュートリアル分野   | 分野横断                                                  |
| 例の複雑さ           | 簡単                                                      |
| 使用SDK              | boto3                                                     |

チュートリアルの最初の部分では、いくつかのAmazonCore Gatewayターゲットを作成します

### チュートリアルアーキテクチャ
このチュートリアルでは、OpenAPI yaml/jsonファイルで定義された操作をMCPツールに変換し、Bedrock AgentCore Gatewayでホストします。
実演目的で、火星の天気に関するクエリに答える火星天気エージェントを構築します。エージェントはNASAのOpen APIを使用します。ソリューションはAmazon BedrockモデルとStrands Agentを使用します。
この例では、火星天気用のgetInsightWeatherツールを持つ非常にシンプルなエージェントを使用します。

## 前提条件

このチュートリアルを実行するには以下が必要です：
* Jupyter notebook（Pythonカーネル）
* uv
* AWS認証情報
* Amazon Cognito

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

In [ ]:
# Amazon SageMakerノートブックを使用しない場合はAWS認証情報を設定
import os
os.environ['AWS_ACCESS_KEY_ID'] = ''
os.environ['AWS_SECRET_ACCESS_KEY'] = ''
os.environ['AWS_DEFAULT_REGION'] = os.environ.get('AWS_REGION', 'us-east-1')

In [ ]:
import os
import sys

# 現在のスクリプトのディレクトリを取得
if '__file__' in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
else:
    current_dir = os.getcwd()  # __file__ が定義されていない場合のフォールバック (例: Jupyter)

# utils.pyが含まれるディレクトリまで移動 (1レベル上)
utils_dir = os.path.abspath(os.path.join(current_dir, '../..'))

# sys.pathに追加
sys.path.insert(0, utils_dir)

# これでutilsをインポートできます
import utils

In [ ]:
#### Gatewayが引き受けるIAMロールを作成
import utils

agentcore_gateway_iam_role = utils.create_agentcore_gateway_role("sample-lambdagateway")
print("Agentcore gateway role ARN: ", agentcore_gateway_iam_role['Role']['Arn'])

# Gatewayへのインバウンド認証用Amazon Cognitoプールを作成

In [ ]:
# Cognito User Poolの作成
import os
import boto3
import requests
import time
from botocore.exceptions import ClientError

REGION = os.environ['AWS_DEFAULT_REGION']
USER_POOL_NAME = "sample-agentcore-gateway-pool"
RESOURCE_SERVER_ID = "sample-agentcore-gateway-id"
RESOURCE_SERVER_NAME = "sample-agentcore-gateway-name"
CLIENT_NAME = "sample-agentcore-gateway-client"
SCOPES = [
    {"ScopeName": "gateway:read", "ScopeDescription": "Read access"},
    {"ScopeName": "gateway:write", "ScopeDescription": "Write access"}
]
scopeString = f"{RESOURCE_SERVER_ID}/gateway:read {RESOURCE_SERVER_ID}/gateway:write"

cognito = boto3.client("cognito-idp", region_name=REGION)

print("Cognitoリソースを作成または取得中...")
user_pool_id = utils.get_or_create_user_pool(cognito, USER_POOL_NAME)
print(f"User Pool ID: {user_pool_id}")

utils.get_or_create_resource_server(cognito, user_pool_id, RESOURCE_SERVER_ID, RESOURCE_SERVER_NAME, SCOPES)
print("リソースサーバが確保されました。")

client_id, client_secret  = utils.get_or_create_m2m_client(cognito, user_pool_id, CLIENT_NAME, RESOURCE_SERVER_ID)
print(f"Client ID: {client_id}")

# ディスカバリURLを取得
cognito_discovery_url = f'https://cognito-idp.{REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration'
print(cognito_discovery_url)

# Gatewayを作成

In [ ]:
# CMKなしでCognito認証機能を持つGatewayを作成します。前のステップで作成したCognito User Poolを使用します
import boto3
gateway_client = boto3.client('bedrock-agentcore-control', region_name = os.environ['AWS_DEFAULT_REGION'])
auth_config = {
    "customJWTAuthorizer": { 
        "allowedClients": [client_id],  # ClientはCognitoで設定されたClientIdと一致する必要があります。例: 7rfbikfsm51j2fpaggacgng84g
        "discoveryUrl": cognito_discovery_url
    }
}
create_response = gateway_client.create_gateway(name='DemoGWOpenAPIAPIKeyNasaOAI',
    roleArn = agentcore_gateway_iam_role['Role']['Arn'], # IAMロールはGateway の作成/一覧/取得/削除権限を持つ必要があります
    protocolType='MCP',
    authorizerType='CUSTOM_JWT',
    authorizerConfiguration=auth_config, 
    description='OpenAPI ターゲットを持つAgentCore Gateway'
)
print(create_response)
# GatewayTarget作成に使用するGatewayIDを取得
gatewayID = create_response["gatewayId"]
gatewayURL = create_response["gatewayUrl"]
print(gatewayID)

# Bedrock AgentCore GatewayによってNASA Open APIをMCPツールに変換

NasaのOpen APIから天気データを取得する火星天気エージェントを作成します。Nasa Insight API[こちら](https://api.nasa.gov/)に登録する必要があります。無料です！登録すると、APIキーがメールで送られてきます。OpenAPIターゲット作成のため認証情報プロバイダーを設定する際にAPIキーを使用してください。

In [ ]:
import boto3
from pprint import pprint
from botocore.config import Config

acps = boto3.client(service_name="bedrock-agentcore-control")

response=acps.create_api_key_credential_provider(
    name="NasaInsightAPIKey",
    apiKey="", # api.nasa.govでサインアップしてAPIキーを取得してください。メールでAPIキーを受け取るまで2分かかります。
)

pprint(response)
credentialProviderARN = response['credentialProviderArn']
pprint(f"エグレス認証情報プロバイダーARN, {credentialProviderARN}")

# OpenAPIターゲットを作成

#### NASA Open API jsonファイルをS3にアップロード

In [ ]:
# S3クライアントを作成
session = boto3.session.Session()
s3_client = session.client('s3')
sts_client = session.client('sts')

# AWSアカウントIDとリージョンを取得
account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
# パラメータを定義
# OpenAPI jsonファイルをアップロードするS3バケット
bucket_name = f'agentcore-gateway-{account_id}-{region}'
file_path = 'openapi-specs/nasa_mars_insights_openapi.json'
object_key = 'nasa_mars_insights_openapi.json'
# put_objectを使用してファイルをアップロードし、レスポンスを読み取り
try:
    if region == "us-east-1":
        s3bucket = s3_client.create_bucket(
            Bucket=bucket_name
        )
    else:
        s3bucket = s3_client.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={
                'LocationConstraint': region
            }
        )
    with open(file_path, 'rb') as file_data:
        response = s3_client.put_object(
            Bucket=bucket_name,
            Key=object_key,
            Body=file_data
        )

    # アカウントIDとリージョンを含むアップロードされたオブジェクトのARNを構築
    openapi_s3_uri = f's3://{bucket_name}/{object_key}'
    print(f'アップロードされたオブジェクトのS3 URI: {openapi_s3_uri}')
except Exception as e:
    print(f'ファイルのアップロードエラー: {e}')

#### アウトバウンド認証を設定してGatewayターゲットを作成

In [ ]:
# OpenAPI仕様ファイル用のS3 URI
nasa_openapi_s3_target_config = {
    "mcp": {
          "openApiSchema": {
              "s3": {
                  "uri": openapi_s3_uri
              }
          }
      }
}

# APIキー認証情報プロバイダー設定
api_key_credential_config = [
    {
        "credentialProviderType" : "API_KEY", 
        "credentialProvider": {
            "apiKeyCredentialProvider": {
                    "credentialParameterName": "api_key", # これを各APIプロバイダーが期待するAPIキー名に置き換えてください。ヘッダーでトークンを渡す場合は、"Authorization"を使用
                    "providerArn": credentialProviderARN,
                    "credentialLocation":"QUERY_PARAMETER", # APIキーの場所。可能な値は "HEADER" と "QUERY_PARAMETER" です。
                    #"credentialPrefix": " " # トークンのプレフィックス。有効な値は "Basic" です。トークンにのみ適用されます。
            }
        }
    }
  ]

targetname='DemoOpenAPITargetS3NasaMars'
response = gateway_client.create_gateway_target(
    gatewayIdentifier=gatewayID,
    name=targetname,
    description='SDKを使用したS3UriによるOpenAPIターゲット',
    targetConfiguration=nasa_openapi_s3_target_config,
    credentialProviderConfigurations=api_key_credential_config)

# Strands AgentからBedrock AgentCore Gatewayを呼び出し

Strands AgentはBedrock AgentCore Gatewayを通じてAWSツールとシームレスに統合され、GatewayはModel Context Protocol（MCP）仕様を実装しています。この統合により、AIエージェントとAWSサービス間での安全で標準化されたコミュニケーションが可能になります。

その核心において、Bedrock AgentCore Gatewayは基本的なMCP API（ListToolsとInvokeTools）を公開するプロトコル準拠のGatewayとして機能します。これらのAPIにより、MCP準拠のクライアントまたはSDKは、安全で標準化された方法で利用可能なツールを発見し、対話することができます。Strands AgentがAWSサービスにアクセスする必要がある場合、これらのMCP標準化エンドポイントを使用してGatewayと通信します。

Gatewayの実装は（MCP承認仕様）[https://modelcontextprotocol.org/specification/draft/basic/authorization]に厳密に準拠しており、堅牢なセキュリティとアクセス制御を確保します。これは、Strands AgentによるすべてのツールInvocationが承認ステップを通過することを意味し、強力な機能を有効にしながらセキュリティを維持します。

例えば、Strands AgentがMCPツールにアクセスする必要がある場合、まずListToolsを呼び出して利用可能なツールを発見し、次にInvokeToolsを使用して特定のアクションを実行します。Gatewayは必要なセキュリティ検証、プロトコル変換、およびサービス相互作用をすべて処理し、全体のプロセスをシームレスで安全にします。

このアーキテクチャアプローチは、MCP仕様を実装する任意のクライアントまたはSDKがGatewayを通じてAWSサービスと対話できることを意味し、AIエージェント統合のための汎用性があり将来に対応したソリューションとなります。

# インバウンド認証用Amazon Cognitoからアクセストークンを要求

In [ ]:
print("Amazon Cognito認証機能からアクセストークンを要求中...ドメイン名の伝播が完了するまでしばらく失敗する場合があります")
token_response = utils.get_token(user_pool_id, client_id, client_secret,scopeString,REGION)
token = token_response["access_token"]
print("トークンレスポンス:", token)

# Bedrock AgentCore Gatewayを使用してNASA Open APIを呼び出すことで火星天気エージェントに質問

In [ ]:
from strands.models import BedrockModel
from mcp.client.streamable_http import streamablehttp_client 
from strands.tools.mcp.mcp_client import MCPClient
from strands import Agent

def create_streamable_http_transport():
    return streamablehttp_client(gatewayURL,headers={"Authorization": f"Bearer {token}"})

client = MCPClient(create_streamable_http_transport)

## ~/.aws/credentialsで設定されたIAMグループ/ユーザーはBedrockモデルへのアクセス権限が必要
yourmodel = BedrockModel(
    model_id="us.amazon.nova-pro-v1:0",
    temperature=0.7,
)

In [ ]:
from strands import Agent
import logging


# ルートstrandsロガーを設定します。問題をデバッグする場合はDEBUGに変更してください。
logging.getLogger("strands").setLevel(logging.INFO)

# ログを表示するためにハンドラーを追加
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

with client:
    # listToolsを呼び出し
    tools = client.list_tools_sync()
    # モデルとツールでAgentを作成
    agent = Agent(model=yourmodel,tools=tools) ## お好きなモデルに置き換えることができます
    print(f"エージェントにロードされたツール: {agent.tool_names}")
    #print(f"エージェントのツール設定: {agent.tool_config}")
    # サンプルプロンプトでエージェントを呼び出し。これはMCP listToolsのみを呼び出し、LLMがアクセスできるツールのリストを取得します。下記では実際にツールを呼び出しません。
    agent("こんにちは、利用可能なすべてのツールをリストしてもらえますか")
    agent("火星の北部地域の天気はどうですか")
    # サンプルプロンプトでエージェントを呼び出し、ツールを呼び出してレスポンスを表示
    #MCPツールを明示的に呼び出し。MCPツール名と引数はAWS Lambda関数またはOpenAPI/Smithy APIと一致する必要があります
    result = client.call_tool_sync(
    tool_use_id="get-insight-weather-1", # これを一意の識別子に置き換えることができます。
    name=targetname+"___getInsightWeather", # これはAWS Lambda ターゲット形式に基づくツール名です。これはターゲット名に応じて変更されます
    arguments={"ver": "1.0","feedtype": "json"}
    )
    #MCPツールレスポンスを印刷
    print(f"ツール呼び出し結果: {result['content'][0]['text']}")

**問題: 下記のセルを実行時に以下のエラーが発生した場合、pydanticとpydantic-coreバージョン間の非互換性を示します。**

```
TypeError: model_schema() got an unexpected keyword argument 'generic_origin'
```
**解決方法は？**

互換性のあるpydantic==2.7.2とpydantic-core 2.27.2を両方確実にインストールする必要があります。完了後にカーネルを再起動してください。

# クリーンアップ
IAMロール、IAMポリシー、認証情報プロバイダー、AWS Lambda関数、Cognitoユーザープール、s3バケットなどの追加リソースもクリーンアップの一環として手動で削除する必要がある場合があります。これは実行する例によって異なります。

## Gatewayを削除（オプション）

In [ ]:
import utils
utils.delete_gateway(gateway_client,gatewayID)